# Methods of Efficiently Solving Recurrence Equations in Python

In [1]:
from itertools import accumulate, chain
import numpy as np

from platform import python_version
python_version()

'3.7.3'

The following problem is based on this stackoverflow question:
- https://stackoverflow.com/q/4407984/1609514

(with arbitrary data created by me)

In [2]:
def t_next(t, data):
    Tm, tau = data  # Unpack more than one data input
    return Tm + (t - Tm)**tau

assert t_next(2, (0.38, 0)) == 1.38

t0 = 2  # Initial t
Tm_values = np.array([0.38, 0.88, 0.56, 0.67, 0.45, 0.98, 0.58, 0.72, 0.92, 0.82])
tau_values = np.linspace(0, 0.9, 10)

## 1. Basic for loop in Python

In [3]:
t = t0
t_out = [t0]
for Tm, tau in zip(Tm_values, tau_values):
    t = t_next(t, (Tm, tau))
    t_out.append(t)
t_out = np.array(t_out)
t_out

array([2.        , 1.38      , 1.81303299, 1.60614649, 1.65039964,
       1.52579703, 1.71878078, 1.66109554, 1.67839293, 1.72152195,
       1.73091672])

## 2. Using Python's built-in accumulate function

In [4]:
# Prepare input data in a 2D array
data_sequence = np.vstack([Tm_values, tau_values]).T

In [5]:
t_out = np.fromiter(accumulate(chain([t0], data_sequence), t_next), dtype=float)
print(t_out)

# Slightly more readable version possible in Python 3.8+
if python_version()[:3] > '3.8':
    t_out = np.fromiter(accumulate(data_sequence, t_next, initial=t0), dtype=float)
    print(t_out)

[2.         1.38       1.81303299 1.60614649 1.65039964 1.52579703
 1.71878078 1.66109554 1.67839293 1.72152195 1.73091672]


In [6]:
def t_next(t, Tm, tau):
    return Tm + (t - Tm)**tau

assert t_next(2, 0.38, 0) == 1.38
assert t_next(1.38, 0.88, 0.1) == 1.8130329915368075

t_next_ufunc = np.frompyfunc(t_next, 3, 1)

assert t_next_ufunc(2, 0.38, 0) == 1.38
assert t_next_ufunc(1.38, 0.88, 0.1) == 1.8130329915368075
assert np.all(t_next_ufunc([2, 1.38], [0.38, 0.88], [0, 0.1]) == [1.38, 1.8130329915368075])

## 3. Using Numpy accumulate method and ufuncs

In [7]:
def test_add(x, data):
    return x + data

assert test_add(1, 2) == 3
assert test_add(2, 3) == 5

# Make a Numpy ufunc from my test_add function
test_add_ufunc = np.frompyfunc(test_add, 2, 1)

assert test_add_ufunc(1, 2) == 3
assert test_add_ufunc(2, 3) == 5
assert np.all(test_add_ufunc([1, 2], [2, 3]) == [3, 5])

data_sequence = np.array([1, 2, 3, 4])
f_out = test_add_ufunc.accumulate(data_sequence, dtype=object)
assert np.array_equal(f_out, [1, 3, 6, 10])

However, I have not found a way to make this work for a function with more than two inputs...

In [23]:
def add_with_power(x, data1, data2):
    return (x + data1) ** data2

assert add_with_power(1, 2, 1) == 3
assert add_with_power(3, 3, 2) == 36

# Make a Numpy ufunc from my test_add function
add_with_power_ufunc = np.frompyfunc(add_with_power, 3, 1)

assert add_with_power_ufunc(1, 2, 1) == 3
assert add_with_power_ufunc(3, 3, 2) == 36
assert np.all(add_with_power_ufunc([1, 3], [2, 3], [1, 2]) == [3, 36])

data_sequence = np.array([1, 2, 3, 4])
try:
    f_out = add_with_power_ufunc.accumulate(data_sequence, dtype=object)
except ValueError as err:
    print(err)

accumulate only supported for binary functions


In [11]:
# Can we trick it by passing more parameters as a tuple?

def add_with_power(x, data):
    return (x + data[0]) ** data[1]

assert add_with_power(1, (2, 1)) == 3
assert add_with_power(3, (3, 2)) == 36

# Make a Numpy ufunc from my test_add function
add_with_power_ufunc = np.frompyfunc(add_with_power, 2, 1)

assert add_with_power_ufunc(1, (2, 1)) == 3
assert add_with_power_ufunc(3, (3, 2)) == 36
assert np.all(add_with_power_ufunc([1, 3], [2, 3], [1, 2]) == [3, 36])

data_sequence = np.array([(2, 1), (3, 2), (4, 3), (5, 4)])
try:
    f_out = add_with_power_ufunc.accumulate(data_sequence, dtype=object)
except ValueError as err:
    print(err)

TypeError: cannot unpack non-iterable int object

In [27]:
two_dim = np.array([
    [1,1,1],
    [2,2,2],
    [3,3,3]
])
np.add.accumulate(two_dim)

array([[1, 1, 1],
       [3, 3, 3],
       [6, 6, 6]])

In [24]:
test_add_ufunc.accumulate(two_dim)

ValueError: could not find a matching type for ? (vectorized).accumulate, requested type has type code 'l'